# Deep Neural Network for MNIST Classification

### Import the relavant packages

In [10]:
import numpy as np  # type: ignore
import tensorflow as tf  # type: ignore
import tensorflow_datasets as tfds # type: ignore

### Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)


In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. 
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)
    
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))





# Model


### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer = 200


 
model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=(28,28,1)),
                              tf.keras.layers.Dense(hidden_layer, activation='relu'),
                              tf.keras.layers.Dense(hidden_layer, activation='tanh'),
                              tf.keras.layers.Dense(hidden_layer, activation='sigmoid'),
                              tf.keras.layers.Dense(output_size, activation='softmax')
     
 ])

C:\Users\Manuel\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Choose the optimizer and lost function

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [8]:
NUM_EPOCHS = 5

early_stopping = tf.keras.callbacks.EarlyStopping()
model.fit(train_data, epochs = NUM_EPOCHS, callbacks=[early_stopping], validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 19s - 34ms/step - accuracy: 0.9127 - loss: 0.3142 - val_accuracy: 0.9548 - val_loss: 0.1449
Epoch 2/5
540/540 - 10s - 18ms/step - accuracy: 0.9671 - loss: 0.1086 - val_accuracy: 0.9707 - val_loss: 0.0963
Epoch 3/5
540/540 - 11s - 20ms/step - accuracy: 0.9781 - loss: 0.0719 - val_accuracy: 0.9772 - val_loss: 0.0754
Epoch 4/5
540/540 - 9s - 16ms/step - accuracy: 0.9836 - loss: 0.0524 - val_accuracy: 0.9852 - val_loss: 0.0523
Epoch 5/5
540/540 - 9s - 16ms/step - accuracy: 0.9873 - loss: 0.0412 - val_accuracy: 0.9868 - val_loss: 0.0429


#### Testing the model

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)


print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'. format(test_loss, test_accuracy*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9750 - loss: 0.0826
Test loss: 0.08. Test accuracy: 97.50%
